In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import BaggingClassifier
from imblearn.metrics import geometric_mean_score
from sklearn.model_selection import StratifiedKFold
import prune
from sklearn.metrics import roc_auc_score
from imblearn.ensemble import RUSBoostClassifier
from sklearn.preprocessing import LabelEncoder
from imblearn.ensemble import EasyEnsembleClassifier
import glob

In [5]:
path ='data' # use your path
allFiles = glob.glob(path + "/*.dat")

In [14]:
file = open('testfile.txt','w') 
 
file.write("“Hello World” \n") 
file.write("“This is our new text file” \n") 
file.write("“and this is another line.”\n") 
file.write("“Why? Because we can.” \n") 
 
file.close()

In [6]:
allFiles

['data/haberman.dat',
 'data/glass-0-1-2-3_vs_4-5-6.dat',
 'data/vehicle1.dat',
 'data/page-blocks0.dat',
 'data/newthyroid2.dat',
 'data/ecoli2.dat',
 'data/glass6.dat',
 'data/wisconsin.dat',
 'data/yeast1.dat',
 'data/vehicle0.dat',
 'data/iris0.dat',
 'data/vehicle2.dat',
 'data/ecoli-0_vs_1.dat',
 'data/glass0.dat',
 'data/glass1.dat',
 'data/vehicle3.dat',
 'data/yeast3.dat',
 'data/ecoli3.dat',
 'data/segment0.dat',
 'data/new-thyroid1.dat',
 'data/ecoli1.dat']

In [11]:
for fileName in allFiles:
    print(fileName)
    data = np.genfromtxt(fname = fileName, comments='@', delimiter=',', autostrip=True)
    X = data[:,:-1]
    print(X[:5])
    data = pd.read_csv(fileName, comment='@', header = None, delimiter=',', delim_whitespace=True)
    y = data.iloc[:,-1].values
    print(y[:5])

data/haberman.dat
[[38. 59.  2.]
 [39. 63.  4.]
 [49. 62.  1.]
 [53. 60.  2.]
 [47. 68.  4.]]
['negative' 'negative' 'negative' 'negative' 'negative']
data/glass-0-1-2-3_vs_4-5-6.dat
[[1.51588824e+00 1.28779500e+01 3.43036000e+00 1.40066000e+00
  7.32820000e+01 6.89310000e-01 8.04468000e+00 0.00000000e+00
  1.22400000e-01]
 [1.51764230e+00 1.29777000e+01 3.53812000e+00 1.21127000e+00
  7.30020000e+01 6.52050000e-01 8.52888000e+00 0.00000000e+00
  0.00000000e+00]
 [1.52212996e+00 1.42079500e+01 3.82099000e+00 4.69760000e-01
  7.17700000e+01 1.11780000e-01 9.57260000e+00 0.00000000e+00
  0.00000000e+00]
 [1.52222108e+00 1.32104500e+01 3.77160000e+00 7.90760000e-01
  7.19884000e+01 1.30410000e-01 1.02452000e+01 0.00000000e+00
  0.00000000e+00]
 [1.51755118e+00 1.33900000e+01 3.65935000e+00 1.18880000e+00
  7.27892000e+01 5.71320000e-01 8.27064000e+00 0.00000000e+00
  5.61000000e-02]]
['negative' 'negative' 'negative' 'negative' 'negative']
data/vehicle1.dat
[[ 95.  48.  83. 178.  72.  10.

In [151]:
enc = LabelEncoder()
y = enc.fit_transform(y)

In [154]:
bagging = EasyEnsembleClassifier(n_estimators= 30)

In [155]:
metrics = np.zeros((2,2))
skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    bagging.fit(X_train, y_train)
    metrics[0,:] += np.array([roc_auc_score(y_test, bagging.predict(X_test)), geometric_mean_score(y_test, bagging.predict(X_test))]) 
    bagging.estimators_ = prune.reduce_error_GM(bagging, X_train, y_train)
    metrics[1,:] += np.array([roc_auc_score(y_test, bagging.predict(X_test)), geometric_mean_score(y_test, bagging.predict(X_test))]) 
metrics /= skf.n_splits
print(metrics)

[[0.84841252 0.83717951]
 [0.86559201 0.85557676]]
